In [1]:
from sentence_transformers import SentenceTransformer
import os
import json
import ast
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

c:\Users\user\anaconda3\envs\easyread\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
model = SentenceTransformer('BAAI/bge-m3')

c:\Users\user\anaconda3\envs\easyread\Lib\site-packages\torch\cuda\__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


KeyboardInterrupt: 

In [8]:
with open('data/documents.txt', 'r', encoding="UTF-8") as f:
    lines = f.readlines()

embedding_list = []
for line in tqdm(lines):
    response = model.encode(line)    
    embedding_list.append(response)  

  0%|          | 0/41754 [00:00<?, ?it/s]

  1%|          | 262/41754 [00:02<07:12, 95.90it/s]


KeyboardInterrupt: 

In [19]:
# 백업
with open('data/miniLM-embedded-vector', 'w', encoding='utf-8') as f:
    f.write(str(embedding_list))

In [ ]:
# 로드
with open('data/miniLM-embedded-vector', 'r', encoding='utf-8') as f:
    embedded_vector = f.read()
    embedding_list = ast.literal_eval(embedded_vector)

print(embedding_list)

In [20]:
em_vectors = []
for i, em in tqdm(enumerate(embedding_list)):
    em_vectors.append({"id": str(i), "values": em, "metadata": {"assertion": lines[i]}})
print(len(em_vectors))

41754it [00:00, 956492.36it/s]

41754


In [22]:
from pinecone import Pinecone

# Pinecone 클라이언트 설정 함수
def init_pinecone_client():
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index = pc.Index("minilm")
    return index

# 업서트 작업을 수행하는 함수
def upsert_batch(batch):
    index = init_pinecone_client()
    index.upsert(vectors=batch)

In [24]:
# 배치 크기 설정
batch_size = 100

# 배치 리스트 생성
batches = [em_vectors[i:i+batch_size] for i in range(0, len(em_vectors), batch_size)]

# ThreadPoolExecutor를 사용한 멀티 스레딩
with ThreadPoolExecutor() as executor:
    # tqdm을 사용하여 진행 상태 표시
    list(tqdm(executor.map(upsert_batch, batches), total=len(batches)))

100%|██████████| 418/418 [03:55<00:00,  1.78it/s]
